In [2]:
import os
import re
import pickle
import datetime
import pandas as pd
import numpy as np

from _html_parser import ParsingDataPrepare, TableConstant

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
%load_ext autoreload
%autoreload 2

#### Choose the date for dirs and files

In [4]:
# Start from more time ('26_05_2022' -'27_03_2021'), ('26_03_2021' - '03_11_2013)
dir_data = dir_data_start = '26_05_2022'
dir_data_end = '27_03_2021'

In [5]:
with open('pickle_files/new_events/'+ dir_data +'/df_'+ dir_data_start +'_to_'+ dir_data_end  +'_games', 'rb') as f:
    df_games = pickle.load(f)

In [6]:
df_games

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals
0,1729359,Ligue 1,18,310,26.05.2022 17:00,Auxerre - Saint-Etienne,1:1
1,1729393,Serie B,627,310,26.05.2022 18:30,Monza - Pisa,2:1
2,1729388,Eredivisie,596,Cup,26.05.2022 18:00,Vitesse - AZ Alkmaar,2:1
3,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0
4,1687458,Eliteserien,747,301,26.05.2022 16:00,Rosenborg - Haugesund,3:3
...,...,...,...,...,...,...,...
91916,15197308,Primera Division,593,306,28.03.2021 17:00,Nueva Concepcion - Suchitepequez,1:1
91917,15197309,Primera Division,593,306,28.03.2021 20:59,Deportivo San Pedro - Plataneros,2:1
91918,15092974,National Football League,575,307,28.03.2021 03:00,Navua - Rewa,1:2
91919,15092975,National Football League,575,307,28.03.2021 03:00,Ba - Lautoka,1:2


In [7]:
df_games['f_com'] = df_games.game_title.map(lambda x: str(x).split(' - ')[0])
df_games['s_com'] = df_games.game_title.map(lambda x: str(x).split(' - ')[1])

In [8]:
df_games.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,f_com,s_com
0,1729359,Ligue 1,18,310,26.05.2022 17:00,Auxerre - Saint-Etienne,1:1,Auxerre,Saint-Etienne
1,1729393,Serie B,627,310,26.05.2022 18:30,Monza - Pisa,2:1,Monza,Pisa
2,1729388,Eredivisie,596,Cup,26.05.2022 18:00,Vitesse - AZ Alkmaar,2:1,Vitesse,AZ Alkmaar


#### Add competitions data 

In [9]:
with open('pickle_files/new_events/'+ dir_data +'/df_comp_tabs_'+ dir_data_start +'_topsec_lgs', 'rb') as f:
    df_comp_tables = pickle.load(f)

In [10]:
df_comp_tables.rename(columns={'team_name':'f_com'}, inplace=True)

In [11]:
# Duble column for get data for each team
df_comp_tables['s_com'] = df_comp_tables.f_com

In [23]:
df_comp_tables

,f_com,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id,league,s_com
0,Red Bull Salzburg,1,22,17,4,1,50,13,+37,55,474,310,top,Red Bull Salzburg
1,Sturm Graz,2,22,10,7,5,46,32,+14,37,474,310,top,Sturm Graz
2,Wolfsberger AC,3,22,11,4,7,34,32,+2,37,474,310,top,Wolfsberger AC
3,Austria Vienna,4,22,8,9,5,31,23,+8,33,474,310,top,Austria Vienna
4,Rapid Vienna,5,22,8,7,7,35,31,+4,31,474,310,top,Rapid Vienna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,Hapoel Haifa,10,26,7,9,10,30,37,-7,30,761,306,sec,Hapoel Haifa
559,Bnei Sakhnin,11,26,8,5,13,15,36,-21,29,761,306,sec,Bnei Sakhnin
560,Hapoel Tel Aviv,12,26,6,9,11,17,28,-11,27,761,306,sec,Hapoel Tel Aviv
561,Hapoel Kfar-Saba,13,26,6,5,15,19,33,-14,23,761,306,sec,Hapoel Kfar-Saba


In [13]:
df_fcom = pd.merge(left=df_games, right=df_comp_tables[['f_com','comp_id','season_id','pos', 'pts']], on=['f_com','comp_id','season_id'], how='left')
df_fcom.rename(columns={'pos':'pos_f', 'pts':'pts_f'}, inplace=True)

In [14]:
df_scom = pd.merge(left=df_games, right=df_comp_tables[['s_com','comp_id','season_id','pos', 'pts']], on=['s_com','comp_id','season_id'], how='left')
df_scom.rename(columns={'pos':'pos_s', 'pts':'pts_s'}, inplace=True)

In [15]:
df_games_tb = pd.merge(df_fcom, df_scom[['pos_s','pts_s']], how='left', left_index=True, right_index=True)

In [16]:
df_games_tb.pos_f = pd.to_numeric(df_games_tb.pos_f, errors='coerce')
df_games_tb.pos_s = pd.to_numeric(df_games_tb.pos_s, errors='coerce')
df_games_tb.pts_f = pd.to_numeric(df_games_tb.pts_f, errors='coerce')
df_games_tb.pts_s = pd.to_numeric(df_games_tb.pts_s, errors='coerce')

In [17]:
df_games_tb['posf_poss'] = df_games_tb[['pos_f', 'pos_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 
df_games_tb['ptsf_ptss'] = df_games_tb[['pts_f', 'pts_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 

In [18]:
df_games_tb.drop(columns=['s_com', 'pos_f', 'pts_f', 'pos_s', 'pts_s'], inplace=True)

In [19]:
df_games_tb.posf_poss.value_counts(dropna=False).head(4)

None     86795
2 - 3       55
1 - 2       51
3 - 1       51
Name: posf_poss, dtype: int64

In [20]:
df_games_tb

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,f_com,posf_poss,ptsf_ptss
0,1729359,Ligue 1,18,310,26.05.2022 17:00,Auxerre - Saint-Etienne,1:1,Auxerre,None,None
1,1729393,Serie B,627,310,26.05.2022 18:30,Monza - Pisa,2:1,Monza,None,None
2,1729388,Eredivisie,596,Cup,26.05.2022 18:00,Vitesse - AZ Alkmaar,2:1,Vitesse,None,None
3,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0,FC Minsk,None,None
4,1687458,Eliteserien,747,301,26.05.2022 16:00,Rosenborg - Haugesund,3:3,Rosenborg,None,None
...,...,...,...,...,...,...,...,...,...,...
91916,15197308,Primera Division,593,306,28.03.2021 17:00,Nueva Concepcion - Suchitepequez,1:1,Nueva Concepcion,None,None
91917,15197309,Primera Division,593,306,28.03.2021 20:59,Deportivo San Pedro - Plataneros,2:1,Deportivo San Pedro,None,None
91918,15092974,National Football League,575,307,28.03.2021 03:00,Navua - Rewa,1:2,Navua,None,None
91919,15092975,National Football League,575,307,28.03.2021 03:00,Ba - Lautoka,1:2,Ba,None,None


In [22]:
# stop

#### Add games events

In [ ]:
with open('pickle_files/new_events/'+ dir_data +'/df_evns_'+ dir_data_start +'_to_'+ dir_data_end, 'rb') as f:
    df_events = pickle.load(f)

In [ ]:
print('shape - ', df_events.shape)
df_events.head(2)

In [ ]:
df_events.event_desc = df_events.event_desc.map(lambda x: [y[0]+y[-1] for y in x])

In [ ]:
df_events.shape

#### Check max len stats_dict col for new names df's columns

In [ ]:
maxlen_stats_dict = df_events.stats_dict.map(lambda x: len(x))
maxlen_stats_dict.max()

In [ ]:
maxlen_stats_dict[maxlen_stats_dict == 19].head(2)

In [ ]:
col_names = [x[0] for x in df_events.stats_dict[2592]]

In [ ]:
str(col_names)

In [ ]:
for col in col_names:
    df_events[col] = df_events.stats_dict.map(lambda x: ''.join([str(y[1]) + '-' + str(y[2]) for y in x if y[0] == col]))

In [ ]:
df_events.drop(columns=['stats_dict'], inplace=True)

In [ ]:
# df_events.head(3)

In [ ]:
# main_ligues = pd.read_csv('csv_files/main_ligues.csv')
# main_ligues = main_ligues.top_ligues.squeeze()
# df_main = df[df.ligue_header.isin(main_ligues)].reset_index(drop=True)
# df_main.shape
# (353909, 8)

In [ ]:
top_ligues = TableConstant.top_ligues
sec_ligues = TableConstant.second_ligues

In [ ]:
df_games_tb['ligue_sts'] = np.where(df_games_tb.comp_id.isin(top_ligues), 'top', np.where(df_games_tb.comp_id.isin(second_ligues),'sec', 'thd'))

#### Data clearing

In [ ]:
df_games_tb.ligue_sts.value_counts()

In [ ]:
df_games_tb = df_games_tb.merge(df_events, left_on='game_id', right_on='game_id')

In [ ]:
sr_coef     = df_games_tb.bet_coefs != ([],[])
sr_pos_pts  = pd.notna(df_games_tb.posf_poss)
df_pos_coef = pd.concat([sr_coef, sr_pos_pts], axis=1)

In [ ]:
mask     = df_pos_coef.bet_coefs == False  
mask_2   = df_pos_coef.posf_poss == False
df_check = df_pos_coef[mask & mask_2]
df_check.shape

In [ ]:
df_games_tb.drop(df_check.index, inplace=True)

In [ ]:
df_games_tb.shape

In [ ]:
df_games_tb.head(1)

#### Add cities and countries

In [ ]:
df_games_tb['city']    = df_games_tb.city_country.map(lambda x: str(x).split(', ')[0] if len(str(x).split(', ')) == 2 else None)
df_games_tb['country'] = df_games_tb.city_country.map(lambda x: str(x).split(', ')[1] if len(str(x).split(', ')) == 2 else None)

In [ ]:
df_games_tb.city.value_counts(dropna=False)[:3]

In [ ]:
with open('pickle_files/df_teams_data', 'rb') as f:
    df_teams_data = pickle.load(f)

In [ ]:
df_teams_data.head(3)

In [ ]:
df_teams_data.rename(columns={'club':'f_com', 'country':'country_tm', 'city':'city_tm'}, inplace=True)

In [ ]:
df_games_tb.f_com = df_games_tb.f_com.map(lambda x: ParsingDataPrepare.str_utf_decode(x))

In [ ]:
df_games_tb = pd.merge(df_games_tb, df_teams_data, how='left', left_on='f_com', right_on='f_com')

In [ ]:
df_games_tb.city.value_counts(dropna=False)[:3]

In [ ]:
sr_city    = df_games_tb[['city', 'city_tm']].apply(lambda x: x[0] if pd.notna(x[0]) else x[1], axis=1)
sr_country = df_games_tb[['country', 'country_tm']].apply(lambda x: x[0] if pd.notna(x[0]) else x[1], axis=1)

In [ ]:
df_games_tb.city    = sr_city
df_games_tb.country = sr_country

In [ ]:
df_games_tb.city.value_counts(dropna=False)[:3]

In [ ]:
df_games_tb.drop(columns=['country_tm', 'city_tm'], inplace=True)

In [ ]:
df_na_cities = pd.DataFrame(df_games_tb[['game_id', 'f_com']][df_games_tb.city.isna()])

In [ ]:
df_na_cities = df_na_cities.drop_duplicates('f_com', keep='first')

In [ ]:
# list_indexes = open('pickle_files/df_game_ids_na_cities', 'wb')
# pickle.dump(df_na_cities, list_indexes)  
# list_indexes.close()

In [ ]:
com_na_cities = df_games_tb.f_com[df_games_tb.city.isna()]
com_na_cities.reset_index(drop=True, inplace=True)

In [ ]:
com_na_cities.value_counts()[:10]

#### Add geo coordinate and cities popylation

In [ ]:
find_c = df_games_tb[['f_com', 'city']].apply(lambda x: x[1] if re.findall('Merseburg', str(x[0])) else None, axis=1)
find_c.value_counts()

In [ ]:
find_c = df_teams_data[['f_com', 'city_tm']].apply(lambda x: x[1] if re.findall('Merseburg', str(x[0])) else None, axis=1)
find_c.value_counts()

In [ ]:
with open('pickle_files/df_world_cities', 'rb') as f:
    df_world_cities = pickle.load(f)

In [ ]:
df_world_cities.head(3)

In [ ]:
sr_city     = pd.notna(df_games_tb.city)
sr_country  = pd.notna(df_games_tb.country)
# df_check = df_pos_coef[sr_city & sr_country]
# df_check.shape

In [ ]:
sr_city.value_counts()

In [ ]:
sr_country.value_counts()

In [ ]:
tmp = df_games_tb.city_country.map(lambda x: str(x).split(', ')[0])

In [ ]:
tmp.value_counts()

In [ ]:
tmp = df_teams_data.f_com.map(lambda x: re.findall('Haig', str(x)))